In [12]:
import pandas as pd
df = pd.read_csv('user_database.csv').set_index('username')
df['password'] = df['password'].astype(str)
users_dict = df.to_dict(orient='index')
df

,password,role,activated
username,,,
admin,111,admin,True


In [15]:
users_dict['admin']['password']

'111'

In [16]:
while True:
    username = input('Please input your username: ')
    if username not in df.index:
        print('Please input valid username')
        continue
    password = input('Please input your password: ')
    if password == users_dict[username]['password']:
        if username == 'admin':
            self.current_user = "adm"
            break
        else:
            self.current_user = 'vol'
            break
    else:
        print('You entered incorrect password')

In [10]:
new

{'username': ['admin'],
 'password': ['111'],
 'role': ['admin'],
 'activated': ['TRUE']}